In [1]:
import pandas as pd

# Scrape Mars Data: The News   .....................  Mod 10.3.3

Automate visiting a website to scrape the top (most recent) news article's Title and Summary.

In [2]:
# Import Splinter and BeautifulSoup                                            

from splinter import Browser
from bs4 import BeautifulSoup as soup       
    # using an alias, "soup," to simplify our code a bit 
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Set up an instance of a Splinter browser

executable_path = {'executable_path': ChromeDriverManager().install()}   
    # Chrome as our browser
browser = Browser('chrome', **executable_path, headless=False)     
    # browser's actions will be displayed in a Chrome window



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\c-hol\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [4]:
# Visit the website

url = 'https://redplanetscience.com'
browser.visit(url)
    # Run this code to make sure it's working correctly. 
    # A new automated browser should open to the featured images webpage.

browser.is_element_present_by_css('div.list_text', wait_time=1)
    # Optional delay for loading the page
    # searching for elements with a specific combination of tag (div) and attribute (list_text)
    # "Browser, wait a second before searching for components." 
    # (useful for dynamic pages that take a little while to load)

True

In [5]:
# Set up the HTML parser:

html = browser.html
news_soup = soup(html, 'html.parser')
    # use BeautifulSoup to parse the HTML (...text and then stores it as an object)

slide_elem = news_soup.select_one('div.list_text')
    # assign slide_elem to look for the <div /> tag and other tags in the <div /> element
    # . is used for selecting classes, such as list_text 
    # div.list_text pinpoints the <div /> tag with the class of list_text

slide_elem

<div class="list_text">
<div class="list_date">February 4, 2022</div>
<div class="content_title">NASA Invites Students to Name Mars 2020 Rover</div>
<div class="article_teaser_body">Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.</div>
</div>

In [6]:
# Assign the title and summary text to variables we'll reference later

slide_elem.find('div', class_='content_title')
    # chained .find to slide_elem to find THIS SPECIFC data (content title) in the ton of data
    # content title is in a <div> with class of 'content_title'

<div class="content_title">NASA Invites Students to Name Mars 2020 Rover</div>

In [7]:
# Use the parent element to find the first `a` tag and save it as `news_title`

news_title = slide_elem.find('div', class_='content_title').get_text()
    # when .get_text() is chained to .find(), only the text of the element is returned

news_title

            # THE RESULT IS THE MOST RECENT TITLE PUBLISHED ON THE WEBSITE

'NASA Invites Students to Name Mars 2020 Rover'

In [8]:
# Get the same thing, but Summary instead of Title

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

news_p

"Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."

# Scrape Mars Data: Featured Image  ...............  Mod 10.3.4

Always pull the full-size featured image: Automate visiting a website, navigating through it, finding the full-size image, extracted a link based on its location on the page.

In [9]:
# Visit URL

url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button

full_image_elem = browser.find_by_tag('button')[1]
    # new variable to hold the scraping result = the browser finds an element by it's tag
    # [1] = indexed to click the second button (ie. 0, 1, 2, etc.; #1 is the second indexed)

full_image_elem.click()                                
    # click the image to view it's size

In [11]:
# Parse the resulting html with soup

html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url

img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
    # "This is where the image we want lives—use the link that's inside these tags."
    # Hey BeautifulSoup, look inside the <img /> tag for an image with a class of fancybox-image
    # An img tag is nested within this HTML, so we've included it.
    # .get('src') pulls the link to the image.

img_url_rel

'image/featured/mars2.jpg'

In [13]:
# Add the base URL to create an full URL

img_url = f'https://spaceimages-mars.com/{img_url_rel}'
    # img_url = holds our f-string.
    # f'___ = f-string to print statements in Python.
    # { } = hold a variable that will be inserted into the f-string when it's executed.

img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Scrape Mars Data: Mars Facts .............................  Mod 10.3.5

In [14]:
# Visit URL

url = 'https://galaxyfacts-mars.com/'
browser.visit(url)

In [15]:
# At the top of your Jupyter Notebook, add import pandas as pd

In [16]:
# scrape the entire table with Pandas' .read_html() function

df = pd.read_html('https://galaxyfacts-mars.com')[0]
    # creating a new DataFrame from the HTML table
    # read_html() = specifically searches for and returns a list of tables found in the HTML
    # an index of 0 = tells Pandas to pull only the 1st table/item it encounters/in the list
    # turns the table into a DataFrame
    
df.columns=['description', 'Mars', 'Earth']
    # assign columns to the new DataFrame for additional clarity
    
df.set_index('description', inplace=True)
    # .set_index() = turns the Description column into the DataFrame's index
    # inplace=True = the updated index will remain in place, 
    # won't have to reassign the DataFrame to a new variable
    
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [17]:
# Turn the table into HTML

df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [18]:
# Close down the website (like turning out the lights when leaving a room)

browser.quit()